In [ ]:
library(tidyverse)
library(data.table)
library(reshape2)
library(ggplot2)
library(RColorBrewer)
library(vegan)
library(dplyr) 
library(ggpubr)
library(Hmisc)
library(corrplot)
library(ggpmisc)
library(scales)
library(broom)
setwd("/home/zaramela/Dropbox/postdoc/skin/eczema_herpeticum/manuscript2021/HMP/")

metadateQiime <- read.delim("EH_metadata_qiime2_forR_v02.tsv", head=T)
rhofile <- read.delim("adsl_rho_data_numeric.csv", h=T)
rhofile <- as.data.frame(rhofile)
#rhofile <- rhofile[-17,]
df <- read.delim("dfratioSpeciesmeta_Malassezia.txt", h=T)
dfgenus <- read.delim("dfratiometa_Genus_Malassezia.txt", h=T)
dfselected <- df[grep("Staphylococcus hominis", df$Species),]
xlabelname <- "log2ratio (Staphylococcus hominis / Malassezia)"
dfselected <- dfselected[which(dfselected$Enrollment != "Yearly Clinical Update"), ]
dfselectedRho <- merge(dfselected, rhofile, by.x = "Subject", by.y = "SUBJID")

meta <- c("ConditionGroup", "Ratio", "EASI", "SCORAD", "VAS", "ITCHD", "IGER", "APRN","INVOPIN_STD")
dfselectedRhometa <- dfselectedRho[,colnames(dfselectedRho) %in% meta]
dfselectedRhometa$logIGER <- log(dfselectedRhometa$IGER)
dfselectedRhometa$logAPRN <- log(dfselectedRhometa$APRN)
options(repr.plot.width=20, repr.plot.height=4)

## Using Tydeverse

analysis <- dfselectedRhometa %>%
  group_by(ConditionGroup) %>%
  nest() %>%
  mutate(model = map(data, ~lm(Ratio ~ logIGER, data = .)),
    cor = map(data, ~tidy(cor.test(.x$Ratio, .x$logIGER, method = "pearson"), 3)))

stats <- analysis %>%
  unnest(cor)

## Ordering groups
         
conditiongroup <- c("Lesional_ADEH+", "Lesional_ADEH-", "Nonlesional_ADEH+", 
                    "Nonlesional_ADEH-", "Non-atopic")

dfselectedRhometa$ConditionGroup <- factor(dfselectedRhometa$ConditionGroup,
                                    levels = conditiongroup, ordered = TRUE)


IgE <- ggplot(dfselectedRhometa, aes(Ratio, logIGER)) +
          geom_point(shape = 20, size = 4) +
          facet_wrap(~ ConditionGroup, ncol = 5) +
          geom_smooth(method = "lm", color="black", se = TRUE, formula = y ~ x) +
          geom_text(data = stats, aes(label = sprintf("r = %s", round(estimate, 3)), x = -7, y = 19), size=5) +
          geom_text(data = stats, aes(label = sprintf("p = %s", round(p.value, 3)),  x = -7, y = 17), size=5) +
          labs(x = xlabelname, y = "IgE") +
          coord_cartesian(ylim = c(-2, 20), xlim = c(-10, 12)) +
          guides(colour = guide_legend(override.aes = list(size=5))) +
          theme_bw() + guides(fill=guide_legend(ncol=1))  +
          theme(axis.text.x=element_text(angle = 0, size = 16, colour = "black"),
                axis.text.y = element_text(angle = 0, size = 16, colour = "black"),
                axis.text = element_text(size = 16, colour = "black"),
                axis.title=element_text(size=16, face = "bold", colour = "black"),
                strip.text.x = element_text(size = 16, face = "bold", colour = "black"),
                strip.background = element_blank(),
                panel.background = element_blank(),
                panel.border = element_rect(colour = "black"),
                panel.grid.major = element_blank(),
                panel.grid.minor = element_blank(),
                axis.line = element_line(colour = "black"),
                legend.position="bottom", legend.box="vertical", legend.margin=margin(), 
                legend.title=element_blank(),
                legend.text=element_text(size=16))
options(repr.plot.width=20, repr.plot.height=4)

## Using Tydeverse

analysis <- dfselectedRhometa %>%
  group_by(ConditionGroup) %>%
  nest() %>%
  mutate(model = map(data, ~lm(Ratio ~ logAPRN, data = .)),
    cor = map(data, ~tidy(cor.test(.x$Ratio, .x$logAPRN, method = "pearson"), 3)))

stats <- analysis %>%
  unnest(cor)

## Ordering groups
         
conditiongroup <- c("Lesional_ADEH+", "Lesional_ADEH-", "Nonlesional_ADEH+", 
                    "Nonlesional_ADEH-", "Non-atopic")

dfselectedRhometa$ConditionGroup <- factor(dfselectedRhometa$ConditionGroup,
                                    levels = conditiongroup, ordered = TRUE)


APRN <- ggplot(dfselectedRhometa, aes(Ratio, logAPRN)) +
          geom_point(shape = 20, size = 4) +
          facet_wrap(~ ConditionGroup, ncol = 5) +
          geom_smooth(method = "lm", color="black", se = TRUE, formula = y ~ x) +
          geom_text(data = stats, aes(label = sprintf("r = %s", round(estimate, 3)), x = -7, y = 19), size=5) +
          geom_text(data = stats, aes(label = sprintf("p = %s", round(p.value, 3)),  x = -7, y = 17), size=5) +
          labs(x = xlabelname, y = "APRN") +
          coord_cartesian(ylim = c(-2, 20), xlim = c(-10, 12)) +
          guides(colour = guide_legend(override.aes = list(size=5))) +
          theme_bw() + guides(fill=guide_legend(ncol=1))  +
          theme(axis.text.x=element_text(angle = 0, size = 16, colour = "black"),
                axis.text.y = element_text(angle = 0, size = 16, colour = "black"),
                axis.text = element_text(size = 16, colour = "black"),
                axis.title=element_text(size=16, face = "bold", colour = "black"),
                strip.text.x = element_text(size = 16, face = "bold", colour = "black"),
                strip.background = element_blank(),
                panel.background = element_blank(),
                panel.border = element_rect(colour = "black"),
                panel.grid.major = element_blank(),
                panel.grid.minor = element_blank(),
                axis.line = element_line(colour = "black"),
                legend.position="bottom", legend.box="vertical", legend.margin=margin(), 
                legend.title=element_blank(),
                legend.text=element_text(size=16)) 
...
options(repr.plot.width=15, repr.plot.height=7)

ggarrange(IgE, APRN,
          labels = c("A", "B"),
          font.label = list(size = 18),
          ncol = 1, nrow = 2)

#ggarrange(IgE, 
#          labels = c("A"),
#          font.label = list(size = 18),
#          ncol = 1, nrow = 1)

options(repr.plot.width=12.3, repr.plot.height=7)

ggarrange(SCORAD, EASI,
          labels = c("C", "D"),
          font.label = list(size = 18),
          ncol = 1, nrow = 2)